In [ ]:
import os
import shutil
import subprocess
import errno

def clone_and_copy(repo_url, branch, destination_dir):
    """
    Clones a Git repository and copies its contents to a specified directory.

    Args:
        repo_url (str): The URL of the Git repository.
        branch (str): The branch to clone.
        destination_dir (str): The directory to copy the contents to.
    """
    # Create the destination directory if it doesn't exist.
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Create a temporary directory for the clone.
    temp_dir = "temp_clone"
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    try:
        # Construct the git clone command.
        clone_command = ["git", "clone", "-b", branch, repo_url, temp_dir]
        # Run the git clone command.
        process = subprocess.Popen(clone_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()

        # Check for errors during cloning.
        if process.returncode != 0:
            print(f"Error cloning repository: {stderr.decode()}")
            return False  # Indicate failure

        # Copy the contents of the cloned repository to the destination directory.
        for item in os.listdir(temp_dir):
            source_path = os.path.join(temp_dir, item)
            dest_path = os.path.join(destination_dir, item)
            if os.path.isdir(source_path):
                #  dirs_exist was introduced in Python 3.8.  This code will work for older versions
                try:
                    shutil.copytree(source_path, dest_path)
                except FileExistsError:
                     # If the directory already exists, copy the files *into* the existing directory
                    for sub_item in os.listdir(source_path):
                        sub_source_path = os.path.join(source_path, sub_item)
                        sub_dest_path = os.path.join(dest_path, sub_item)
                        if os.path.isdir(sub_source_path):
                            shutil.copytree(sub_source_path, sub_dest_path)
                        else:
                            shutil.copy2(sub_source_path, sub_dest_path)
                except Exception as e:
                    print(f"Error copying directory {source_path} to {dest_path}: {e}")
                    return False
            else:
                shutil.copy2(source_path, dest_path)  # copy2 preserves metadata

        print(f"Successfully cloned and copied contents to {destination_dir}")
        return True #Indicate Success

    except Exception as e:
        print(f"An error occurred: {e}")
        return False # Indicate Failure
    finally:
        # Remove the temporary directory.  Important to clean up!
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)



if __name__ == "__main__":
    repo_url = "https://github.com/ruslanmv/watsonx-agent-client.git"
    branch = "dev"
    destination_dir = "/content"
    success = clone_and_copy(repo_url, branch, destination_dir)
    if success:
        print(f"Repository cloned and contents copied to: {destination_dir}")
    else:
        print(f"Failed to clone and copy repository.")


In [ ]:
import os
import subprocess
import sys
import time

def is_google_colab():
    """Checks if the code is running in a Google Colab environment."""
    return 'google.colab' in sys.modules

def run_command(command, check=True, cwd=None, verbose=True, env=None):
    """
    Executes a shell command and optionally checks for errors.

    Args:
        command (list or str): The command to execute as a list of arguments, or a string.
        check (bool, optional): If True, raises a CalledProcessError on non-zero exit. Defaults to True.
        cwd (str, optional):  The current working directory in which to run the command.
        verbose (bool, optional): If True, prints the command and its output in real-time. Defaults to True.
        env (dict, optional): Additional environment variables to pass to the command. Defaults to None.

    Returns:
        tuple: (stdout, stderr) as strings. Returns empty strings if there is an error.
    Raises:
        subprocess.CalledProcessError: If the command fails (and check is True).
    """
    if isinstance(command, str):
        command = [command]

    process_env = os.environ.copy()
    if env:
        process_env.update(env)

    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
                                     cwd=cwd, bufsize=1, env=process_env)

        if verbose:
            print(f"Executing: '{' '.join(command)}'")

        stdout_output, stderr_output = "", ""
        while True:
            if process.stdout:
                line = process.stdout.readline()
                if line:
                    if verbose:
                        print(f"Stdout: {line.strip()}")
                    stdout_output += line
            if process.stderr:
                line = process.stderr.readline()
                if line:
                    if verbose:
                        print(f"Stderr: {line.strip()}")
                    stderr_output += line
            if process.poll() is not None:
                break

        if check and process.returncode != 0:
            error_message = f"Command failed with exit code {process.returncode}:\nStdout:\n{stdout_output}\nStderr:\n{stderr_output}"
            print(error_message)
            raise subprocess.CalledProcessError(process.returncode, command, output=stdout_output, stderr=stderr_output)

        return stdout_output, stderr_output

    except subprocess.CalledProcessError as e:
        print(f"Command execution error: {e}")
        return "", str(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "", str(e)

def create_script(script_name, script_content):
    """Creates a shell script with the given name and content."""
    try:
        script_path = os.path.join(os.getcwd(), script_name)
        with open(script_path, "w") as f:
            f.write(script_content)
        os.chmod(script_path, 0o755)
        print(f"✅ {script_name} successfully saved at: {script_path}")
        return script_path
    except Exception as e:
        print(f"❌ Failed to create {script_name}: {e}")
        return None

def create_install_colab_sh():
    """Creates the install_colab.sh script."""
    script_content = """#!/bin/bash

set -eux

# Check if running in Colab based on the COLAB_ENV variable passed from Python
if [[ "$COLAB_ENV" == "true" ]]; then
    echo "Detected Google Colab. Skipping Python 3.11 install."
    IS_COLAB=true
else
    echo "Not Colab. Proceeding with Python 3.11 install."
    IS_COLAB=false
fi

if [ "$IS_COLAB" = "false" ]; then
    echo "🔧 Adding deadsnakes PPA..."
    sudo add-apt-repository ppa:deadsnakes/ppa -y
    sudo apt update
    echo "🐍 Installing Python 3.11..."
    sudo apt install -y python3.11 python3.11-venv python3.11-distutils python3-apt
    echo "⚙️ Setting Python 3.11 as default python3..."
    sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
    sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
    sudo update-alternatives --config python3
    echo "💡 Installing pip for Python 3.11..."
    curl -sS https://bootstrap.pypa.io/get-pip.py | sudo python3.11
    echo "✅ Python version after setup:"
    python3 --version
    python3 -m pip --version
fi

if [ -d ".venv" ]; then
    echo ".venv exists. Activating..."
else
    echo "Creating .venv..."
    python3 -m venv .venv
    if [ $? -ne 0 ]; then
        echo "❌ Error creating .venv."
        exit 1
    fi
fi

source .venv/bin/activate
echo "⬆️ Upgrading pip..."
pip install --upgrade pip

if [ -f "requirements.txt" ]; then
    echo "📦 Installing requirements..."
    pip install -r requirements.txt
else
    echo "📄 requirements.txt not found."
fi

echo "🎉 Base environment setup complete."

for script in install_beeai.sh install_langflow.sh install_watsonx_sdk.sh install_langraph.sh; do
    if [ -f "$script" ]; then
        echo "▶️ Running $script"
        bash "$script"
    else
        echo "$script not found. Skipping."
    fi
done

echo "🎉 All environments setup triggered."
exit 0
"""
    return create_script("install_colab.sh", script_content)

def main():
    """
    Checks the environment and executes the appropriate install script.
    """
    try:
        if is_google_colab():
            print("Detected Google Colab environment.")
            script_path = create_install_colab_sh()
            if script_path:
                print("Executing install_colab.sh...")
                run_command(["bash", script_path], verbose=True, check=True, env={"COLAB_ENV": "true"})
                print("install_colab.sh executed successfully.")
        else:
            print("Not detected as Google Colab.")
            print("Please ensure 'install.sh' exists and run it manually if needed.")
            # You could add logic here to create a basic install.sh if required
            # or guide the user on manual execution.
            # Example of creating a basic install.sh:
            install_sh_content = """#!/bin/bash
echo "This is a basic install.sh. Please customize it for your local environment."
echo "You might want to install Python dependencies here."
if [ -f "requirements.txt" ]; then
    echo "Installing requirements..."
    python3 -m venv .venv
    source .venv/bin/activate
    pip install --upgrade pip
    pip install -r requirements.txt
else
    echo "requirements.txt not found."
fi
"""
            create_script("install.sh", install_sh_content)
            print("A basic 'install.sh' has been created. Please review and run it manually.")

    except subprocess.CalledProcessError as e:
        print(f"Error during installation: {e}")
        sys.exit(1)
    except Exception as e:
        print(f"An unexpected error occurred in main: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Detected Google Colab environment.
✅ install_colab.sh successfully saved at: /content/install_colab.sh
Executing install_colab.sh...
Executing: 'bash /content/install_colab.sh'
Stdout: Detected Google Colab. Skipping Python 3.11 install.
Stderr: + [[ true == \t\r\u\e ]]
Stdout: .venv exists. Activating...
Stderr: + echo 'Detected Google Colab. Skipping Python 3.11 install.'
Stdout: ⬆️ Upgrading pip...
Stderr: + IS_COLAB=true
Stdout: Requirement already satisfied: pip in ./.venv/lib/python3.11/site-packages (25.0.1)
Stderr: + '[' true = false ']'
Stdout: 📦 Installing requirements...
Stderr: + '[' -d .venv ']'
Stdout: Requirement already satisfied: langchain==0.3.10 in ./.venv/lib/python3.11/site-packages (from -r requirements.txt (line 1)) (0.3.10)
Stderr: + echo '.venv exists. Activating...'
Stdout: Requirement already satisfied: python-dotenv>=0.20.0 in ./.venv/lib/python3.11/site-packages (from -r requirements.txt (line 2)) (1.1.0)
Stderr: + source .venv/bin/activate
Stdout: Requirem